In [1]:
import glob
from tqdm.notebook import trange, tqdm
import scipy.io as sio

# run `jupyter nbextension enable --py widgetsnbextension` in terminal once before

In [2]:
x = []
y = []
info = []

mat_files = glob.glob('data/raw_data/*.mat')
print (len(mat_files))

for mat_file in tqdm(mat_files):
    print (mat_file)
    test = sio.loadmat(mat_file, squeeze_me=True)['TEST']
    control = test[0]
    pws = control['PWs']
    pw_data = control['PW_DATA']
    for pw_index in range(len(pws)):
        currents = pw_data[pw_index]['CURRENTS']
        current_data = pw_data[pw_index]['CURRENT_DATA']
        for current_index in range(len(currents)):
            label = (pws[pw_index], currents[current_index])
            stim_data = current_data[current_index]['STIM_DATA']
            for stim_index in range(len(stim_data)):
                vagus_window = stim_data[stim_index]['VAGUS_WINDOW']
                if(len(vagus_window) != 0):
                    x.append(vagus_window)
                    y.append(label)
                    info.append(mat_file)

15


data/raw_data/Test_s100316_Tests1  2  3  4.mat
data/raw_data/Test_s110216_Tests1  2  3  4.mat
data/raw_data/Test_s050416_Tests1  2  3  4.mat
data/raw_data/Test_s250516_Tests2  4  5  6  7.mat
data/raw_data/Test_s260416_Tests2  3  4  5_x.mat
data/raw_data/Test_s310516_Tests1  2  3  4.mat
data/raw_data/Test_s200416_Tests1  2  3  4.mat
data/raw_data/Test_s020316_Tests3  4.mat
data/raw_data/Test_s210316_Tests1  2  3  4.mat
data/raw_data/Test_s070416_Tests3  5  6  7.mat
data/raw_data/Test_s200416_Tests1  2  3  5.mat
data/raw_data/Test_s220416_Tests1  2  3  4.mat
data/raw_data/Test_s160316_Tests1  2  3  4.mat
data/raw_data/Test_s260416_Tests2  3  4  5.mat
data/raw_data/Test_s210316_Tests1  2  3  5.mat



In [3]:
import numpy as np

x = np.vstack(x)
y = np.vstack(y)

# np.savetxt('data/training_data/time_series.csv', x, delimiter=',', fmt='%1.4f')
# np.savetxt('data/training_data/labels.csv', y, delimiter=',', fmt='%1.4f')

In [5]:
# plotting code

target = [1, 2]
indexes = (y == target).all(axis=1).nonzero()
results = x[indexes]

import matplotlib.pyplot as plt

for result in results:
    plt.plot(result)

(125, 4001)


In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

In [21]:
# split into multiple outputs for simultaneous classification

pw_train, current_train = np.hsplit(y_train, 2)
pw_test, current_test = np.hsplit(y_test, 2)

In [22]:
import tensorflow as tf

print(tf.__version__)
print(tf.keras.__version__)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.1.0
2.2.4-tf
Num GPUs Available:  1


In [23]:
import datetime
import os

batch_size = 128
epochs = 1000

model_name = str(datetime.datetime.now()).split('.')[0]
model_dir = os.path.join('models', model_name)

os.makedirs(model_dir, exist_ok=True)

checkpoint_format = '{epoch:02d}-{val_pw_output_accuracy:.2f}-{val_current_output_accuracy:.2f}.hdf5'
checkpoint_fn = os.path.join(model_dir, checkpoint_format)

log_fn = os.path.join('logs', model_name)

callbacks = []

callbacks.append(tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_fn, verbose=1, save_best_only=True))
callbacks.append(tf.keras.callbacks.EarlyStopping(min_delta=0.0002, patience=40))
callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, min_lr=0.0001))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=log_fn))

In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM

# good god
input_shape = x_train[np.newaxis][:,0,:].T.shape
input_layer = Input(shape=input_shape)
lstm = LSTM(32)(input_layer)

dense = Dense(16, activation='relu')(lstm)

dense_pw = Dense(8, activation='relu', name="pw_dense")(dense)
output_pw = Dense(1, name="pw_output")(dense_pw)

dense_current = Dense(8, activation='relu', name="current_dense")(dense)
output_current = Dense(1, name="current_output")(dense_current)

model = Model(inputs=input_layer, outputs=[output_pw, output_current])

In [25]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy', 'mean_squared_error'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 4001, 1)]    0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 32)           4352        input_3[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           528         lstm_2[0][0]                     
__________________________________________________________________________________________________
pw_dense (Dense)                (None, 8)            136         dense_2[0][0]                    
____________________________________________________________________________________________

In [26]:
x_train_expanded = np.expand_dims(x_train, axis=-1)
x_test_expanded = np.expand_dims(x_test, axis=-1)

In [27]:
from keras_tqdm import TQDMNotebookCallback

callbacks.append(TQDMNotebookCallback())
model.fit(
    x_train_expanded, [pw_train, current_train],
    epochs=epochs, batch_size=batch_size,
    validation_data=(x_test_expanded, [pw_test, current_test]),
    callbacks=callbacks,
    verbose=0)


Epoch 00001: val_loss improved from inf to 2.24006, saving model to models/2020-01-22 15:44:45/01-0.22-0.07.hdf5



Epoch 00002: val_loss improved from 2.24006 to 2.15747, saving model to models/2020-01-22 15:44:45/02-0.22-0.07.hdf5



Epoch 00003: val_loss improved from 2.15747 to 2.15247, saving model to models/2020-01-22 15:44:45/03-0.23-0.07.hdf5



Epoch 00004: val_loss did not improve from 2.15247


KeyError: 'val_pw_output_accuracy'